<!--
<img src="https://drive.google.com/uc?id=1kPqSyZUHR_CxjN8MdQXI3nfOtcRqE_Le" alt="REDE RPPS" height="62" width="52">
-->

<center>

#  <font color="blue">**ANÁLISE DE FLUXO ATUARIAL - GERAÇÃO ATUAL**</font>

</center>






<br>

---

 NOME DO ENTE: **XXX - XX**   
 ANO: **2020**    
 ATUÁRIO RESPONSÁVEL: **XXX** - MIBA **XXX**

<br>

 Elaborado por: **XXX**    
 Data de Elaboração: **XXX**

---

<br>

---

# **IDENTIFICAÇÃO DO FLUXO ATUARIAL**

---


In [ ]:
#@title **Dados do Fluxo Atuarial:**
NOME_ARQUIVO <- "2019_FLX_CIVIL_PREV_GA_XXXXXXXXXXXXXX.csv" #@param {type: "string"}

# #@markdown <br>

# #@markdown **Definição do ente e ano do DRAA correspondente ao Fluxo Atuarial em exame**
CNPJ_ENTE <- "" #@param {type: "string"}
ANO_DRAA  <-  "2019"  #@param ['2022','2021','2020', '2019', '2018', '2017', '2016', '2015']    
TX_JUROS_ANO_ANTERIOR <-   0.0#@param {type: "number"}

TIPO_PLANO  <-  "Previdenci\xE1rio"  #@param ['Previdenciário', 'Financeiro', 'Militar', 'Tesouro']  
TIPO_PLANO <- iconv(TIPO_PLANO, from="latin1", to="utf-8")


# Imprime valores para conferência
NOME_ARQUIVO
CNPJ_ENTE
ANO_DRAA
TX_JUROS_ANO_ANTERIOR
TIPO_PLANO

In [ ]:
#@title **Informações sobre o Fluxo Atuarial** 
#@markdown **_Informações para comparar com o Relatório de Avaliação Atuarial_**

suppressPackageStartupMessages(library(readr))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(readxl))
suppressPackageStartupMessages(library(scales))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(repr))
suppressPackageStartupMessages(library(lubridate))
suppressPackageStartupMessages(library(purrr))
options(scipen=99,
        repr.plot.width=12,
        repr.plot.height=7,
        warn = -1,
        digits = 15)


#===============================================================================
# Importação dos dados

dados <- paste("/content/", NOME_ARQUIVO, sep="")

flx_orig <- read_csv2(dados,
                      skip = 5,
                      col_names = FALSE,
                      locale=locale(decimal_mark = ",", grouping_mark = "."),
                      col_types = cols())

# Colocar nomes nas colunas
colfluxo <- c(100101, 100201, 100301, 100401, 109001,	111000,	111101,	111201, 111301,
              111401,	112000,	119900,	121000,	121100,	121200,	121300,	121400,	121500,
              121600,	121700,	122000, 122100,	122200,	122300,	122400,	122500,	122600,
              122700,	123000,	123100,	123200,	123300,	123400, 124000,	124100,	124200,
              124300,	124400,	124500,	129000,	130101,	130201,	139901,	190000,	210000,
              211001,	212001,	213001,	214001,	215001,	219901,	220000,	221000,	222000,
              223000,	224000,	225000, 226000,	227000,	229000,	239901,	240000,	250001,
              260001,	270001,	280001,	290001)


# Excluir colunas vazias
flx_orig <- flx_orig %>%
              select(1:length(colfluxo))



# Verfica se o fluxo atuarial tem calculo de duration ou não 
# não é necessáro em tese, visto que só o modelo sem duration deve ser encaminhado á SPREV
if(ncol(flx_orig) == 67){
  names(flx_orig) <- paste("V", colfluxo, sep="")
} else { 
  names(flx_orig) <- paste("V", c(colfluxo, 310000,	320000), sep="")
}


# Reter as linhas de controle "Totais de Controle" e "Totais de Controle a Valor Atual"
linhas_controle <- filter(flx_orig, !grepl("^\\d+", V100101))

# e excluí-las da base de dados
flx_orig <- filter(flx_orig, grepl("^\\d+", V100101))


#===============================================================================
# Checagens

juros <- unique(flx_orig$V100301)
tipo_fluxo <- if(flx_orig$V100101[1] == 0){"Antecipado"} else {"Postecipado"}
ano_inicio <- flx_orig$V100201[1]



#===============================================================================
# Cálculo da duração do passivo

Fi <- flx_orig %>%
        mutate(Fi = V210000 + V220000 - V219901 - V229000 - V111000 - V123000 - V124000) %>%
        pull(Fi)

if(tipo_fluxo == "Postecipado"){
    #BenefLiq <- Fi / (1 + flx_orig$V100301 / 100) ^ (as.integer(flx_orig$V100101) - 0.5)
    BenefLiq <- Fi / (1 + TX_JUROS_ANO_ANTERIOR / 100) ^ (as.integer(flx_orig$V100101) - 0.5)
    BenefLiqPond <- BenefLiq * (as.integer(flx_orig$V100101) - 0.5)
} else {
    # Cálculo para o fluxo antecipado: soma-se 0.5 em vez de diminuir
    #BenefLiq <- Fi / (1 + flx_orig$V100301 / 100) ^ (as.integer(flx_orig$V100101) + 0.5)
    BenefLiq <- Fi / (1 + TX_JUROS_ANO_ANTERIOR / 100) ^ (as.integer(flx_orig$V100101) + 0.5)  
    BenefLiqPond <- BenefLiq * (as.integer(flx_orig$V100101) + 0.5)
}

duracao_passivo <- round(sum(BenefLiqPond) / sum(BenefLiq), 2)

#===============================================================================
# Cálcuclo do ìndice de Cobertura - IC

ativos <-  flx_orig$V290001[1] - flx_orig$V250001[1] - flx_orig$V280001[1]

provisao <- linhas_controle$V240000[2] -
            linhas_controle$V190000[2] +
            linhas_controle$V130101[2] +
            linhas_controle$V130201[2] +
            linhas_controle$V139901[2]

ICP <- round(ativos / provisao * 100, 2)


#===============================================================================
# Cálculo da alíquota de contribuição praticada - servidor ativo

aliquota_contrib <- unique(round(flx_orig$V122000/ flx_orig$V109001 * 100, 2))
aliquota_contrib <- aliquota_contrib[!is.na(aliquota_contrib)]

#===============================================================================
# Cálculo da alíquota de contribuição praticada - ente

aliquota_contrib_ente <- unique(round(flx_orig$V121000/ flx_orig$V109001 * 100, 2))
aliquota_contrib_ente <- aliquota_contrib[!is.na(aliquota_contrib_ente)]

#===============================================================================
# Cálculo das variáveis

vabf <- apply(linhas_controle[2, c("V210000", "V220000")], 1, sum, na.rm=TRUE)
vacf <- apply(linhas_controle[2, c("V111000", "V112000", "V119900", "V121000", "V122000", "V123000", "V124000", "V129000", "V130101", "V130201", "V139901")], 1, sum, na.rm=TRUE)
ativos_garantidores <-  flx_orig$V290001[1] - flx_orig$V250001[1] - flx_orig$V280001[1]
rmbc <- linhas_controle[2, "V210000"] - apply(linhas_controle[2, c("V111000", "V112000", "V119900")], 1, sum, na.rm=TRUE)
rmbac <- linhas_controle[2, "V220000"] - apply(linhas_controle[2, c("V121000", "V122000", "V123000", "V124000", "V129000")], 1, sum, na.rm=TRUE)
parcelamentos <- linhas_controle[2, "V130201"]
plano_amortizacao <- linhas_controle[2, "V130101"]


# Impressão dos resultados
cat("Taxa de Juros: ", juros, "\n")
cat("Tipo de Fluxo:", tipo_fluxo, "\n")
cat("Ano de Início do Fluxo:", ano_inicio, "\n")
cat("Duração do Passivo:", duracao_passivo, "anos\n")
cat("Índice de Cobertura do Passivo:", ICP, "%", "\n")
cat("Alíquota Contribuição do Servidor Ativo:", aliquota_contrib, "%", "\n")
cat("Alíquota Contribuição do Ente:", aliquota_contrib_ente, "%", "\n")

cat("===========================================================================\n")

cat("VABF - Valor Atual dos Benefícios Futuros:", prettyNum(vabf, big.mark = ".", decimal.mark = ","), "\n")
cat("VACF - Valor Atual das Contribuições Futuras:", prettyNum(vacf, big.mark = ".", decimal.mark = ","), "\n")
cat("Ativos Garantidores do Plano:", prettyNum(ativos_garantidores, big.mark=".", , decimal.mark = ","), "\n")
cat("PMP - Provisão Matemática Previdenciária:", prettyNum(vabf - vacf, big.mark = ".", decimal.mark = ","), "\n")
cat("Resultado Atuarial:", prettyNum((vacf + ativos) - vabf, big.mark = ".", decimal.mark = ","), "\n")
cat("RMBC - Reservas Matemáticas de Benefícios Condedidos:", prettyNum(rmbc, big.mark = ".", decimal.mark = ","), "\n")
cat("RMBaC - Reservas Matemáticas de Benefícios a Conceder:", prettyNum(rmbac, big.mark = ".", decimal.mark = ","), "\n")
cat("Parcelamentos:", prettyNum(parcelamentos, big.mark = ".", decimal.mark = ","), "\n")
cat("Plano de Amortização:", prettyNum(plano_amortizacao, big.mark = ".", decimal.mark = ","), "\n")



**Considerações:**    


<br>

---

# **PARTE I - ANÁLISE DE RAZOABILIDADE**

---

A análise de razoabilidade consiste em verificar a adequação das "curvas" de algumas variáveis do fluxo atuarial. 

In [ ]:
#@title **Gráfico 1 - Base de Cálculo da Contribuição Normal (109001)**
flx_orig[1:75,] %>%
    ggplot(aes(x=V100201, y=V109001)) +
    geom_line(size=1.3, color="blue") +
    labs(title = "Evolução da Base de Cálculo da Contribuição Normal",
         subtitle = NOME_ARQUIVO,
         x = "Anos",
         y = "Base de Cálculo Contrib. Normal") +
    theme_bw(base_size = 15) #+
    #theme(axis.text=element_text(size=14),
    #      axis.title=element_text(size=14,face="bold"))

**CONSIDERAÇÕES:** 

In [ ]:
#@title **Gráfico 2 - Despesas com os Aposentados Atuais (211001 + 212001 + 213001 + 214001)**
flx_orig[1:75,] %>%
    mutate(APOSENTADOS = V211001 + V212001 + V213001 + V214001) %>%
    ggplot(aes(x=V100201, y=APOSENTADOS)) + 
    geom_line(size=1.3, color="blue") +
    labs(title = "Evolução das despesas com aposentados",
         subtitle = NOME_ARQUIVO,
         x = "Anos",
         y = "Despesas com aposentados") +
    theme_bw(base_size = 15)

**CONSIDERAÇÕES:** 

In [ ]:
#@title **Gráfico 3 - Despesas com os Pensionistas Atuais (215001)**
flx_orig[1:75,] %>%
  ggplot(aes(x=V100201, y=V215001)) +
    geom_line(size=1.3, color="blue") +
    labs(title = "Evolução das despesas com pensionistas",
         subtitle = NOME_ARQUIVO,
         x = "Anos",
         y = "Despesas com pensionistas") +
    theme_bw(base_size = 15)

**CONSIDERAÇÕES:** 

In [ ]:
#@title **Gráfico 4 - Despesas com os Aposentados Futuros (221000 + 222000 + 223000 + 224000)**
flx_orig[1:75,] %>%
  mutate(APOSENTADOS_FUTUROS = V221000 + V222000 + V223000 + V224000) %>%
  ggplot(aes(x=V100201, y=APOSENTADOS_FUTUROS)) +
    geom_line(size=1.3, color="blue") +
    labs(title = "Evolução das despesas com aposentados",
         subtitle = NOME_ARQUIVO,
         x = "Anos",
         y = "Despesas com aposentados") +
    theme_bw(base_size = 15)

**CONSIDERAÇÕES:** 

In [ ]:
#@title **Gráfico 5 - Despesas com os Pensionistas Futuros (225000 + 226000)**
flx_orig[1:75,] %>%
  mutate(PENSIONISTAS_FUTUROS = V225000 + V226000) %>%
  ggplot(aes(x=V100201, y=PENSIONISTAS_FUTUROS)) +
    geom_line(size=1.3, color="blue") +
    labs(title = "Evolução das despesas com pensionistas",
         subtitle = NOME_ARQUIVO,
         x = "Anos",
         y = "Despesas com pensionistas") +
    theme_bw(base_size = 15)

**CONSIDERAÇÕES:** 

In [ ]:
#@title **Gráfico 6 - Receitas (190000) e Despesas (240000) Totais**
# Colocar as duas curvas no mesmo gráfico.
flx_orig[1:75,] %>%
    select(V100201, V190000, V240000) %>%
    pivot_longer(-V100201, names_to="VARIAVEL", values_to = "VALOR") %>%
    ggplot(aes(x=ymd(V100201, truncated=2L), y=VALOR, color=VARIAVEL)) +
      geom_line(size=1.5) +
      labs(title = "Evolução das Receitas e Despesas Totais",
         subtitle = NOME_ARQUIVO,
         x = "Anos",
         y = "Valores das Receitas e Despesas") + 
    theme_bw(base_size = 15) +
    theme(axis.text.x = element_text(angle = 90)) +
    scale_x_date(date_breaks="5 year", date_labels = "%Y") + 
    scale_color_manual(labels = c("Receita", "Despesa"),
                       values = c("blue", "tomato")) 
      


**CONSIDERAÇÕES:** 

In [ ]:
#@title **Gráfico 7 - Gráfico dos Encargos - Ativos, Aposentados e Pensionistas (atuais e futuros)**

#Altera o tamanho dos gráficos
options(repr.plot.width=15, repr.plot.height=8)

flx_orig[1:75,] %>%
    mutate(FOLHA_ATIVOS         = V109001,
           APOSENTADOS_ATUAIS   = V211001 + V212001 + V213001 + V214001,
           PENSIONISTAS_ATUAIS  = V215001,
           APOSENTADOS_FUTUROS  = V221000 + V222000 + V223000 + V224000,
           PENSIONISTAS_FUTUROS = V225000 + V226000) %>%
    select(V100201, FOLHA_ATIVOS, APOSENTADOS_ATUAIS, PENSIONISTAS_ATUAIS, APOSENTADOS_FUTUROS, PENSIONISTAS_FUTUROS) %>%
    pivot_longer(-V100201, names_to="VARIAVEL", values_to = "VALOR") %>%
    ggplot(aes(x=ymd(V100201, truncated=2L), y=VALOR, color=VARIAVEL)) +
      geom_line(size=1.5) +
      labs(title = "Encargos com Ativos, Aposentados e Pensionistas",
         subtitle = NOME_ARQUIVO,
         x = "Anos",
         y = "Valores") + 
    theme_bw(base_size = 15) +
    theme(axis.text.x = element_text(angle = 90)) +
    scale_x_date(date_breaks="5 year", date_labels = "%Y") #+ 
    #scale_color_discrete(labels = c("Folha Ativos", "Aposentados Futuros", "Pensionistas Atuais", "Aposentados Futuros", "Pensionistas Futuros")) 

# Volta à configuração original
options(repr.plot.width=12, repr.plot.height=7)

**CONSIDERAÇÕES:**  

<br>

<br>

---

# **PARTE II - CONFERÊNCIA DAS LINHAS DE CONTROLE DO FLUXO ATUARIAL**

---

<br>


In [ ]:
#@title **Resultado da Comparação das Linhas de Controle**
#@markdown O quadro abaixo evidencia a comparação entre o valor presente das variáveis no fluxo atuarial original, isto é, conforme encaminhado pelo RPPS à SPREV (`soma_flxorigin_vp`) e o valor presente dessas mesmas variáveis, porém recalculadas conforme planilha modelo disponibilizada pela SPREV (`soma_flxrecalc_vp`). 

#@markdown As eventuais diferenças encontradas constam da variável `dif_vp`  

#===============================================================================
# Baixa o arquivo contendo o dicionário de dados
system('git clone https://github.com/marcosfs2006/rede_rpps.git')
setwd("/content/rede_rpps")


#===============================================================================
# Define uma função auxiliar para calcular as taxas de desconto

calcula_tx_desconto <- function(i, postecipado = TRUE){
  # i: Valor informado na variável V100301 - Taxa de Juros (%)
  if(postecipado){
    desconto <- 1 / (1 + i)
  } else {
    desconto <- c(1, 1 / (1 + i[2:length(i)])) 
  }
  round(cumprod(desconto), 5)
}

# Identifica se o fluxo é postecipado ou não.
postecipado <- ifelse(flx_orig$V100101[1] == 1, TRUE, FALSE)

# Obtém o ano do fluxo (ano inicial do fluxo...)
ano <- flx_orig$V100201[1]

#===============================================================================
# Recálculo de algumas variáveis...
flx_orig <- mutate(flx_orig,
                  
                  # fator de desconto
                  V100401 = calcula_tx_desconto(i = flx_orig$V100301 / 100, postecipado = postecipado), 
                  
                  # Benefícios Concedidos - Contribuições dos Aposentados
                  V111000 = V111101 + V111201 + V111301 + V111401 ,
                  
                  # Benefícios a Conceder - Contribuições do Ente
                  V121000 = V121100 + V121200 + V121300 + V121400 + V121500 + V121600 + V121700,
                  
                  # Benefícios a Conceder - Contribuições dos Segurados Ativos
                  V122000 = V122100 + V122200 + V122300 + V122400 + V122500 + V122600 + V122700,
                  
                  # Benefícios a Conceder - Contribuições dos Aposentados
                  V123000 = V123100 + V123200 + V123300 + V123400,
                  
                  # Benefícios a Conceder - Contribuição dos Pensionistas
                  V124000 = V124100 + V124200 + V124300 + V124400 + V124500,
                  
                  # Total das Receitas com Contribuições e Compensação Previdenciária
                  V190000 = V111000 + V112000 + V119900 + V121000 + V122000 + V123000 + V124000 + V129000 + V130101 + V130201 + V139901,
                  
                  # Benefícios Concedidos - Encargos
                  V210000 = V211001 + V212001 + V213001 + V214001 + V215001 + V219901,
                  
                  # Benefícios a Conceder - Encargos
                  V220000 = V221000 + V222000 + V223000 + V224000 + V225000 + V226000 + V227000 + V229000 + V239901,
                  
                  # Total das Despesas com Benefícios do Plano
                  V240000 = V210000 + V220000,
                  
                  # Insuficiência ou Excedente Financeiro
                  V250001 = V190000 - V240000)

#==============================================================
# Codigos a serem conferidos:
codigos <- c("V109001", "V111000", "V111101", "V111201", "V111301", "V111401", "V112000",
             "V119900", "V121000", "V121100", "V121200", "V121300", "V121400", "V121500",
             "V121600", "V121700", "V122000", "V122100", "V122200", "V122300", "V122400",
             "V122500", "V122600", "V122700", "V123000", "V123100", "V123200", "V123300",
             "V123400", "V124000", "V124100", "V124200", "V124300", "V124400", "V124500",
             "V129000", "V130101", "V130201", "V139901", "V190000", "V210000", "V211001",
             "V212001", "V213001", "V214001", "V215001", "V219901", "V220000", "V221000",
             "V222000", "V223000", "V224000", "V225000", "V226000", "V227000", "V229000",
             "V239901", "V240000", "V250001")

#==============================================================
# Função para o cálculo do valor presente das variáveis

vp <- function(x) sum(x * flx_orig$V100401, na.rm = TRUE)


total_flxrecalc <- flx_orig %>%
                      select(all_of(codigos)) %>%
                      map_dbl(sum, na.rm=TRUE) 

total_flxrecalc_vp <- flx_orig %>%
                        select(all_of(codigos)) %>%
                        map_dbl(vp)

controle_flxrecalc <- bind_rows(total_flxrecalc,
                                total_flxrecalc_vp)


#==============================================================
# Obtenção das variáveis de interesse na variável "linhas_controle"

controle_flxorigin <- as_tibble(linhas_controle[, codigos])

#==============================================================
# Importa o dicionário de dados

dicionario <- read_csv2("VariaveisFluxoAtuarial.csv",
                         locale=locale(encoding="latin1",
                         decimal_mark = ",",
                         grouping_mark = "."), col_types = cols())

dicionario <- dicionario %>%
                mutate(codigo = paste("V", codigo, sep=""))

#===============================================================================
                # junta as duas bases de dados
controle_flx <- bind_rows(controle_flxorigin,
                          controle_flxrecalc) %>%
                
                # faz a transposição e converte para tibble
                t() %>%
                as_tibble(rownames=NA) %>%
                          
                # renomeia as colunas do data frame
                rename(soma_flxorigin    = V1,
                       soma_flxorigin_vp = V2,
                       soma_flxrecalc    = V3,
                       soma_flxrecalc_vp = V4) %>%
                
                # insere na base duas novas colunas: codigo e dif_vp
                mutate(codigo = rownames(.),
                       dif_vp = round(soma_flxorigin_vp - soma_flxrecalc_vp))

# Cria um novo data frame apenas com os linhas onde existe diferenças
controle_flx2 <- controle_flx %>%                
                
                # descarta os registros em que a diferença é nula
                filter(dif_vp != 0) %>%

                # junta à base os nomes das variáveis
                inner_join(dicionario, by="codigo") %>%

                # seleciona as colunas de interesse
                select(codigo, nome, soma_flxorigin_vp, soma_flxrecalc_vp, dif_vp)

# Prepara o resultado
if(nrow(controle_flx2) == 0){
  "NÃO FORAM IDENTIFICADAS DIVERGÊNCIAS!"
} else {
  knitr::kable(controle_flx2, digits=2, format.args=list(big.mark = ".", decimal.mark =",") ) 
}


**CONSIDERAÇÕES:** 